In [226]:
# import modules
import numpy as np
import pandas as pd
import os
from scipy.linalg import lstsq # To increase efficiency, and double check the results
import sympy as sp
import re
np.set_printoptions(precision=5,linewidth=100) # adjust setting for prettier printing
import tkinter as tk
import tkinter.filedialog
# %gui tk
# create a gui to let user select the working folder
# root=tk.Tk()
# working_folder = tkinter.filedialog.askdirectory(parent=root,title='Please select a directory')
# root.destroy() # close the gui after user selects the input 
# print(working_folder)

In [227]:
working_folder='E:/Ender/coursework/photogrammetry/Ass4'

In [228]:
#extract files from the working folder
file_list=os.listdir(working_folder)
for i in file_list:
    if i.endswith('.icf'):
        icf=i # extract icf filename
    elif i.endswith('.prj'):
        prj=i # extract prj filename
    elif i.endswith('.ori'):
        ori=i # extract ori filename
    elif i.endswith('.cam'):
        cam=i # extract cam filename
    elif i.endswith('.gcp'):
        gcp=i       

In [229]:
os.chdir(working_folder) # change the path to working_folder
# Read .prj file to get the max iteration times and threshold for stopping the iteration
with open(prj,'r') as f:
    txt=f.read() 
    txt=np.array(re.split('>>|\ |\n',txt)) # split the text so that I can index the value.
max_iteration=np.int32(txt[-7]) # Max iteration times
maxSigma=np.float32(txt[-6]) # The maximum allowable difference between unit weight variances in successive iterations. If the difference is less than this value, then the iterations will stop.
print('max_iteration:', max_iteration)
print('maxSigma:', maxSigma)

# Read .icf to get xy (image coordinates)
xy_img=pd.read_csv(icf,sep='\t|\   |\  ',engine='python',names=['Image_ID','Point_ID','x','y','w1','w2','w3','w4'])
## Subset the xy for each image
xy_img_04=xy_img[xy_img['Image_ID']=='65_04_20180803.jpg'] # subset the dataframe to get xy for 65_09_20180803.jpg
xy_img_05=xy_img[xy_img['Image_ID']=='65_05_20180803.jpg'] # repeat the process for the other images.
xy_img_11=xy_img[xy_img['Image_ID']=='65_11_20180803.jpg']
xy_img_12=xy_img[xy_img['Image_ID']=='65_12_20180803.jpg']
# Read IOP based on the output from lab 1
with open(cam,'r') as f:
    txt=f.read() 
    txt=np.array(re.split('\t|\n',txt)) # split the text so that I can index the value.
xp,yp,c=np.float32(txt[9:12]) # extract xp,yp,c

print(f'xp={xp}, yp={yp}, c={c}')

k1,k2,k3,p1,p2=np.float32(txt[22:27]) #extract distortion parameters
print(f'k1={k1}, k2={k2}, k3={k3}, p1={p1}, p2={p2}')
# Read GCP from .gcp
gcp=pd.read_csv(gcp,engine='python',sep='\t|\   ',names=['Point_ID','X','Y','Z'],usecols=[0,1,2,3])
X_gcp=np.array(gcp['X']) # extract x of gcp
Y_gcp=np.array(gcp['Y'])
Z_gcp=np.array(gcp['Z'])
# Read .ori as the input EOP
with open(ori,'r') as f:
    for i in f:
        text=i
        if text.startswith('65_04_20180803'):
            text=(re.split(',',text))
            text=text[1::]
            text=np.array(text).astype(np.float32)
            omg_04,phi_04,kpp_04,Xo_04,Yo_04,Zo_04=text[0:6]
        elif text.startswith('65_05_20180803'):
            text=(re.split(',',text))
            text=text[1::]
            text=np.array(text).astype(np.float32)
            omg_05,phi_05,kpp_05,Xo_05,Yo_05,Zo_05=text[0:6]
        elif text.startswith('65_11_20180803'):
            text=(re.split(',',text))
            text=text[1::]
            text=np.array(text).astype(np.float32)
            omg_11,phi_11,kpp_11,Xo_11,Yo_11,Zo_11=text[0:6]
        elif text.startswith('65_12_20180803'):
            text=(re.split(',',text))
            text=text[1::]
            text=np.array(text).astype(np.float32)
            omg_12,phi_12,kpp_12,Xo_12,Yo_12,Zo_12=text[0:6]
print(omg_05,phi_05,kpp_05,Xo_05,Yo_05,Zo_05)

max_iteration: 200
maxSigma: 1e-12
xp=0.06722940504550934, yp=-0.11779332906007767, c=8.15868854522705
k1=-0.0002753081498667598, k2=8.035393875616137e-06, k3=-2.082014987081493e-07, p1=6.32564042462036e-05, p2=-7.169081800384447e-05
7.5549345 -12.209332 1.2043861 1.2821908 1.2642989 3.7040656


In [230]:
# angle conversion
def deg2rad(angle): return np.deg2rad(np.float32(angle)) # When I read the angles from the text files, they are string, so I need to convert the type first.
def rad2deg(angle): return np.rad2deg(angle) # convert radian to degree
def preprocessing(xy_img):
    '''
    This function is used to pre-process the data for LSA
    '''
    num_observation=len(xy_img['x']) #compute number of observation
    xx=np.array(xy_img['x']).astype(np.float32) # define x_img
    yy=np.array(xy_img['y']).astype(np.float32) # define y_img
    point_ID=np.array(xy_img['Point_ID']).astype(np.int16)
    # The following lines compute the distortion
    x_bar=xx-xp
    y_bar=yy-yp
    rd=np.sqrt(x_bar**2+y_bar**2) # distance from principle point
    dist_x=x_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p1*(rd**2+2*x_bar**2)+2*p2*x_bar*y_bar
    dist_y=y_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p2*(rd**2+2*y_bar**2)+2*p1*x_bar*y_bar
    # the following lines compute x_corrected and y_corrected
    x_corrected=xx-dist_x
    y_corrected=yy-dist_y
    return x_corrected,y_corrected, num_observation,point_ID
def get_rotation_matrix(omg,phi,kpp):
    R_omg = np.array([[1,0,0],[0,np.cos(omg),-np.sin(omg)],[0,np.sin(omg),np.cos(omg)]]) # rotation matrix for omega
    R_phi = np.array([[np.cos(phi),0,np.sin(phi)],[0,1,0],[-np.sin(phi),0,np.cos(phi)]]) # rotation matrix for phi
    R_kpp = np.array([[np.cos(kpp),-np.sin(kpp),0],[np.sin(kpp),np.cos(kpp),0],[0,0,1]]) # rotation matrix for kappa
    R=R_omg@R_phi@R_kpp # Final rotation matrix
    return R

In [231]:
# xy_img_04['Point_ID']=xy_img_04['Point_ID'].astype(int)
print((list(xy_img_04['Point_ID'])[0]))
print(omg_04,phi_04,kpp_04)

5
-15.391508 -9.605362 1.4864464


## Linear two-light Intersection

In [232]:
def linear_two_light_intersection(omg_l,phi_l,kpp_l,Xo_l,Yo_l,Zo_l,omg_r,phi_r,kpp_r,Xo_r,Yo_r,Zo_r,xy_img_l,xy_img_r,X_gcp,Y_gcp,Z_gcp):
    omg_l,phi_l,kpp_l=deg2rad(omg_l),deg2rad(phi_l),deg2rad(kpp_l) # convert angles to radian
    omg_r,phi_r,kpp_r=deg2rad(omg_r),deg2rad(phi_r),deg2rad(kpp_r) # convert angles to radian
    R_l=get_rotation_matrix(omg_l,phi_l,kpp_l)
    R_r=get_rotation_matrix(omg_r,phi_r,kpp_r)
    Xo_l,Yo_l,Zo_l=np.float64(Xo_l),np.float64(Yo_l),np.float64(Zo_l) # convert type to float
    Xo_r,Yo_r,Zo_r=np.float64(Xo_r),np.float64(Yo_r),np.float64(Zo_r) # convert type to float
    B=np.array([Xo_r-Xo_l,Yo_r-Yo_l,Zo_r-Zo_l]) # Define B matrix
    tie_points_ID=set(xy_img_l['Point_ID'].astype(np.int16)).intersection(set(xy_img_r['Point_ID'].astype(np.int16))) # find the tie points ID
    tie_points_ID=np.array(list(tie_points_ID)) # Convert tie points ID from set to list in order to index it
    xy_img_l_subset=xy_img_l[xy_img_l['Point_ID'].astype(int).isin(tie_points_ID)]
    xy_img_r_subset=xy_img_r[xy_img_r['Point_ID'].astype(int).isin(tie_points_ID)]
    x_corrected_l,y_corrected_l, num_observation_l,point_ID_l=preprocessing(xy_img_l_subset)
    x_corrected_r,y_corrected_r, num_observation_r,point_ID_r=preprocessing(xy_img_r_subset)
    XYZ_hat=[]
    for i in range(len(tie_points_ID)):
        n=tie_points_ID[i]

        v_l=np.array([x_corrected_l[i]-xp,y_corrected_l[i]-yp,-c])
        v_r=np.array([x_corrected_r[i]-xp,y_corrected_r[i]-yp,-c])
        A=np.array([R_l@v_l, -R_r@v_r]).T
        dx,sigma_new,rank,s=lstsq(A,B)
        X_l=Xo_l+(dx[0]*R_l@v_l)[0]
        Y_l=Yo_l+(dx[0]*R_l@v_l)[1]
        Z_l=Zo_l+(dx[0]*R_l@v_l)[2]
        X_r=Xo_r+(dx[1]*R_r@v_r)[0]
        Y_r=Yo_r+(dx[1]*R_r@v_r)[1]
        Z_r=Zo_r+(dx[1]*R_r@v_r)[2]
        X_avg=(X_l+X_r)/2
        Y_avg=(Y_l+Y_r)/2
        Z_avg=(Z_l+Z_r)/2
        XYZ_hat.append([X_avg,Y_avg,Z_avg])
    XYZ_hat=np.array(XYZ_hat)
    # print(X_hat.shape)
    tie_points_ID_temp=tie_points_ID-1
    X_gcp_subset=[X_gcp[i] for i in tie_points_ID_temp]
    Y_gcp_subset=[Y_gcp[i] for i in tie_points_ID_temp]
    Z_gcp_subset=[Z_gcp[i] for i in tie_points_ID_temp]
    dx=XYZ_hat[:,0]-X_gcp_subset
    dy=XYZ_hat[:,1]-Y_gcp_subset
    dz=XYZ_hat[:,2]-Z_gcp_subset
    RMSE_x=np.sqrt(np.sum(dx**2)/(len(tie_points_ID)-1))
    RMSE_y=np.sqrt(np.sum(dy**2)/(len(tie_points_ID)-1))
    RMSE_z=np.sqrt(np.sum(dz**2)/(len(tie_points_ID)-1))
    RMSE_xyz=[RMSE_x,RMSE_y,RMSE_z]
    print(RMSE_x,RMSE_y,RMSE_z)
    return XYZ_hat, RMSE_xyz


XYZ_hat_set1, RMSE_xyz_set1=linear_two_light_intersection(omg_04,phi_04,kpp_04,Xo_04,Yo_04,Zo_04,omg_05,phi_05,kpp_05,Xo_05,Yo_05,Zo_05,xy_img_04,xy_img_05,X_gcp,Y_gcp,Z_gcp)
XYZ_hat_set2, RMSE_xyz_set2=linear_two_light_intersection(omg_11,phi_11,kpp_11,Xo_11,Yo_11,Zo_11,omg_12,phi_12,kpp_12,Xo_12,Yo_12,Zo_12,xy_img_11,xy_img_12,X_gcp,Y_gcp,Z_gcp)


0.03761303674125118 0.01366858028386314 0.02592255364138327
0.039361025487138296 0.011775485236213959 0.028351925030496626


## Linear Multiple-Light-Ray Intersection

In [244]:
EOPs_04=[omg_04,phi_04,kpp_04,Xo_04,Yo_04,Zo_04]
EOPs_05=[omg_05,phi_05,kpp_05,Xo_05,Yo_05,Zo_05]
EOPs_11=[omg_11,phi_11,kpp_11,Xo_11,Yo_11,Zo_11]
EOPs_12=[omg_12,phi_12,kpp_12,Xo_12,Yo_12,Zo_12]
EOPs_11

[-0.25732535, -16.414362, 92.72951, 1.0135577, 2.1563983, 3.0377772]

In [243]:
def Multiple_light_ray_intersection(EOPs_A,EOPs_B,EOPs_C,EOPs_D,xy_img_A,xy_img_B,xy_img_C,xy_img_D,X_gcp,Y_gcp,Z_gcp):
    omg_A,phi_A,kpp_A=deg2rad(EOPs_A[0]),deg2rad(EOPs_A[1]),deg2rad(EOPs_A[2]) # convert angles to radian
    omg_B,phi_B,kpp_B=deg2rad(EOPs_B[0]),deg2rad(EOPs_B[1]),deg2rad(EOPs_B[2]) # convert angles to radian
    omg_C,phi_C,kpp_C=deg2rad(EOPs_C[0]),deg2rad(EOPs_C[1]),deg2rad(EOPs_C[2]) # convert angles to radian
    omg_D,phi_D,kpp_D=deg2rad(EOPs_D[0]),deg2rad(EOPs_D[1]),deg2rad(EOPs_D[2]) # convert angles to radian
    R_A,R_B,R_C,R_D=get_rotation_matrix(omg_A,phi_A,kpp_A),get_rotation_matrix(omg_B,phi_B,kpp_B),get_rotation_matrix(omg_C,phi_C,kpp_C),get_rotation_matrix(omg_D,phi_D,kpp_D)
    Xo_A,Yo_A,Zo_A=np.float64(EOPs_A[3]),np.float64(EOPs_A[4]),np.float64(EOPs_A[5])
    Xo_B,Yo_B,Zo_B=np.float64(EOPs_B[3]),np.float64(EOPs_B[4]),np.float64(EOPs_B[5])
    Xo_C,Yo_C,Zo_C=np.float64(EOPs_C[3]),np.float64(EOPs_C[4]),np.float64(EOPs_C[5])
    Xo_D,Yo_D,Zo_D=np.float64(EOPs_D[3]),np.float64(EOPs_D[4]),np.float64(EOPs_D[5])

    tie_points_ID_temp1=set(xy_img_A['Point_ID'].astype(np.int16)).intersection(set(xy_img_B['Point_ID'].astype(np.int16))) # find the tie points ID
    tie_points_ID_temp2=set(xy_img_C['Point_ID'].astype(np.int16)).intersection(set(xy_img_D['Point_ID'].astype(np.int16))) # find the tie points ID
    tie_points_ID=list(tie_points_ID_temp1.intersection(tie_points_ID_temp2))
    X_gcp_subset=[X_gcp[i] for i in tie_points_ID]
    Y_gcp_subset=[Y_gcp[i] for i in tie_points_ID]
    Z_gcp_subset=[Z_gcp[i] for i in tie_points_ID]
    xy_img_A_subset=xy_img_A[xy_img_A['Point_ID'].astype(int).isin(tie_points_ID)]
    xy_img_B_subset=xy_img_B[xy_img_B['Point_ID'].astype(int).isin(tie_points_ID)]
    xy_img_C_subset=xy_img_C[xy_img_C['Point_ID'].astype(int).isin(tie_points_ID)]
    xy_img_D_subset=xy_img_D[xy_img_D['Point_ID'].astype(int).isin(tie_points_ID)]
    x_corrected_A,y_corrected_A, num_observation_A,point_ID_A=preprocessing(xy_img_A_subset)
    x_corrected_B,y_corrected_B, num_observation_B,point_ID_B=preprocessing(xy_img_B_subset)
    x_corrected_C,y_corrected_C, num_observation_C,point_ID_C=preprocessing(xy_img_C_subset)
    x_corrected_D,y_corrected_D, num_observation_D,point_ID_D=preprocessing(xy_img_D_subset)
    XYZ_hat=[]
    for i in range(len(tie_points_ID)):
        v_A=np.array([x_corrected_A[i]-xp,y_corrected_A[i]-yp,c]).T
        v_B=np.array([x_corrected_B[i]-xp,y_corrected_B[i]-yp,c]).T
        v_C=np.array([x_corrected_C[i]-xp,y_corrected_C[i]-yp,c]).T
        v_D=np.array([x_corrected_D[i]-xp,y_corrected_D[i]-yp,c]).T
        [Nx_A,Ny_A,D_A]=R_A@v_A
        [Nx_B,Ny_B,D_B]=R_B@v_B
        [Nx_C,Ny_C,D_C]=R_C@v_C
        [Nx_D,Ny_D,D_D]=R_D@v_D
        A=np.array([[D_A,0,-Nx_A],
                    [0,D_A,-Ny_A],
                    [D_B,0,-Nx_B],
                    [0,D_B,-Ny_B],
                    [D_C,0,-Nx_C],
                    [0,D_C,-Ny_C],
                    [D_D,0,-Nx_D],
                    [0,D_D,-Ny_D]])
        y=np.array([D_A*Xo_A-Nx_A*Zo_A,
                    D_A*Yo_A-Ny_A*Zo_A,
                    D_B*Xo_B-Nx_B*Zo_B,
                    D_B*Yo_B-Ny_B*Zo_B,
                    D_C*Xo_C-Nx_C*Zo_C,
                    D_C*Yo_C-Ny_C*Zo_C,
                    D_D*Xo_D-Nx_D*Zo_D,
                    D_D*Yo_D-Ny_D*Zo_D,]).T
        X_h,sigma_new,rank,s=lstsq(A,y)
        XYZ_hat.append(X_h)
    XYZ_hat=np.array(XYZ_hat)
    dx=XYZ_hat[:,0]-X_gcp_subset
    dy=XYZ_hat[:,1]-Y_gcp_subset
    dz=XYZ_hat[:,2]-Z_gcp_subset
    RMSE_x=np.sqrt(np.sum(dx**2)/(len(tie_points_ID)-1))
    RMSE_y=np.sqrt(np.sum(dy**2)/(len(tie_points_ID)-1))
    RMSE_z=np.sqrt(np.sum(dz**2)/(len(tie_points_ID)-1))
    RMSE_xyz=[RMSE_x,RMSE_y,RMSE_z]
    print(RMSE_x,RMSE_y,RMSE_z)
    return XYZ_hat, RMSE_xyz


Multiple_light_ray_intersection(EOPs_04,EOPs_05,EOPs_11,EOPs_12,xy_img_04,xy_img_05,xy_img_11,xy_img_12,X_gcp,Y_gcp,Z_gcp)


1.4600259016653108 1.5311680295319 0.9838090411159732


(array([[0.86437, 1.05346, 1.39582],
        [2.76555, 1.31494, 0.9415 ],
        [2.00623, 1.30502, 0.72907],
        [1.29011, 1.37581, 0.81238],
        [0.77681, 1.48218, 1.07472],
        [2.84032, 2.09965, 0.72075],
        [2.0257 , 2.07939, 0.56805],
        [1.28792, 2.06383, 0.70706],
        [0.77005, 2.05384, 1.02055],
        [2.69174, 2.83511, 0.95975],
        [1.97389, 2.79656, 0.81422],
        [1.31171, 2.69504, 0.94725],
        [0.85085, 2.57065, 1.25232]]),
 [1.4600259016653108, 1.5311680295319, 0.9838090411159732])